In [7]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from typing import List, Tuple
import multiprocessing as mp
from functools import partial

def determine_column_types(chunk: pd.DataFrame) -> dict:
    column_types = {}
    for column in chunk.columns:
        if pd.api.types.is_string_dtype(chunk[column]):
            column_types[column] = 'string'
        elif pd.api.types.is_numeric_dtype(chunk[column]):
            column_types[column] = 'numeric'
        else:
            column_types[column] = 'unknown'
    return column_types

def compare_columns(col1: pd.Series, col2: pd.Series) -> float:
    if pd.api.types.is_string_dtype(col1) and pd.api.types.is_string_dtype(col2):
        vectorizer = HashingVectorizer(n_features=1000)
        vec1 = vectorizer.fit_transform(col1.astype(str).fillna(''))
        vec2 = vectorizer.transform(col2.astype(str).fillna(''))
        vec1 = np.asarray(vec1.mean(axis=0))
        vec2 = np.asarray(vec2.mean(axis=0))
        return cosine_similarity(vec1, vec2)[0][0]
    elif pd.api.types.is_numeric_dtype(col1) and pd.api.types.is_numeric_dtype(col2):
        return abs(pearsonr(col1.fillna(0), col2.fillna(0))[0])
    else:
        return 0.0

def process_chunk(chunk1: pd.DataFrame, df2: pd.DataFrame) -> List[Tuple[str, str, float]]:
    similar_columns = []
    for col1 in chunk1.columns:
        for col2 in df2.columns:
            similarity = compare_columns(chunk1[col1], df2[col2])
            if similarity > 0.8:
                similar_columns.append((col1, col2, similarity))
    return similar_columns

def main():
    file1 = 'SampleData1.csv'
    file2 = 'SampleData2.csv'

    print("Loading second file...")
    df2 = dd.read_csv(file2).compute()

    print("Loading first file in chunks...")
    df1 = dd.read_csv(file1, blocksize="10MB")

    print("Determining column types...")
    types1 = determine_column_types(df1.head(1))
    types2 = determine_column_types(df2)

    print("Column types for Sample 1:")
    for col, type in types1.items():
        print(f"{col}: {type}")

    print("Column types for Sample 2:")
    for col, type in types2.items():
        print(f"{col}: {type}")

    print("Comparing columns...")

    pool = mp.Pool(mp.cpu_count())
    similar_columns = []

    for delayed_chunk in df1.to_delayed():
        chunk = delayed_chunk.compute()
        chunk_results = pool.apply_async(process_chunk, args=(chunk, df2))
        similar_columns.extend(chunk_results.get())

    pool.close()
    pool.join()

    similar_columns = sorted(set(similar_columns), key=lambda x: x[2], reverse=True)

    print("Similar columns:")
    for col1, col2, similarity in similar_columns:
        print(f"{col1} (Sample 1) and {col2} (Sample 2) - Similarity: {similarity:.2f}")

if __name__ == "__main__":
    main()


Loading second file...
Loading first file in chunks...
Determining column types...
Column types for Sample 1:
First Name: string
Last Name: string
Mobile Number: numeric
Date: string
Email: string
Address: string
Column types for Sample 2:
Email: string
Account Balance: string
User ID: string
Meter No: string
Meter Reading: numeric
Comparing columns...
Similar columns:
Email (Sample 1) and Email (Sample 2) - Similarity: 1.00


In [5]:
import pandas as pd

def process_csv(csv_file):
    df = pd.read_csv(csv_file)

    rows, columns = df.shape
    print(f'Number of Rows: {rows}')
    print(f'Number of Columns: {columns}')

    required_columns = ['First Name', 'Last Name','Mobile Number']
    if not all(col in df.columns for col in required_columns):
        print("Error: CSV file does not contain the required columns.")
        return

    print("\nNames and Phone Numbers:")
    for _, row in df[required_columns].iterrows():
        print(f"Name: {row['First Name']} {row['Last Name']}, Phone Number: {row['Mobile Number']}")


csv_file = 'SampleData1.csv'
process_csv(csv_file)

Number of Rows: 6
Number of Columns: 6

Names and Phone Numbers:
Name: John Saint, Phone Number: 1234567890
Name: Abraham Lincoln, Phone Number: 9876543021
Name: Rahim Mohammed, Phone Number: 8765123490
Name: Harry Potter, Phone Number: 4321567089
Name: Bruce Wayne, Phone Number: 6789043215
Name: Kara ZorEl, Phone Number: 1234509876
